<h1 align="center">Cloud Computing</h1>

<a id='Cloud Platforms'></a>
<h3 style="background-color:#616161;color:white">1. Cloud Platforms</h3>

<b>Five principle of cloud/utility computing:</b>
* Shared Resources: Cost: capital vs. operating expenses
* Virtualization: “infinite” capacity
* Elasticity: scale up or down on demand
* Automation
* Metered Billing

<b>Value of utility computing (i.e. On Demand computing), in addition to above</b>
* Lower barrier to entry for tackling big-data problems
* Commoditization and democratization of big-data capabilities

<b>When does the cloud make sense?</b>
* Limited lifetime requirement or short-term need
* Scale variability or volatility
* Non-strategic applications or low organizational value

<b>Cloud operating models</b>
* Infrastructure as a Service (IaaS)
    * Why buy machines when you can rent cycles?
    * Examples: Amazon EC2, Rackspace
* Platform as a Service (PaaS)
    * Give me nice API and take care of the maintenance, upgrades, …
    * Example: Google App Engine (GAE)
* Software as a Service (SaaS)
    * Just run it for me!
    * Example: Gmail, Salesforce’s Online CRM

<a id='Distributed Systems'></a>
<h3 style="background-color:#616161;color:white">2. Distributed Systems</h3>

<b>RESTful API</b>
* one way of providing interoperability between computer systems on the internet
* the user progresses through an application by selecting links (state transitions), resulting in the next page (representing the next state of the application) being transferred to the user and rendered for their use
* REST-compliant Web services allow requesting systems to access and manipulate textual representations of Web resources using a uniform and predefined set of stateless operations

<b>HTTP Requests:</b>
* GET method should be safe (nullipotent) - calling it produces no side effects (unlike PUT and GET which may change things)
* Allow requesting systems to access and manipulate textual representations of Web resources using a uniform and predefined set of stateless operations.
* The GET, PUT and DELETE methods should be idempotent - repeated calls should have the same effect as a single request 
* Post is not idempotent
* By making use of a stateless protocol and standard operations, REST systems aim for fast performance, reliability, and the ability to grow, by re-using components that can be managed and updated without affecting the system as a whole, even while it is running


<h4>Lamport Timestamps</h4>

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud001.png" height="100" width="500">

How the timestamp (i.e. integer incrementing) works:
* Everything is set to 0 to begin with
* Everything can do 3 things: Internal action, Send, & Recieve.
* Whenver internal or send happens it increments its own timestamp by 1
* Whenever something recieves something it takes the max of its own timestamp and that of the recieved message, and increments that by 1 to get to a new timestamp number for itself

Notes:
* Does not allow global ordering of event
* Lamport lacks Gap Detection - cannot tell if an event exists elsewhere that sits between two events
* Can however assume that if an event from a comes before event from b, then a's clock is lower than b's clock (Known as the Clock consistency condition)
* The strong clock consistency condition assumes this and vice-versa

<h4> Deadlocks</h4>

A deadlock is a situation in which two or more competing actions are each waiting for the other to finish, and thus neither ever does

The Cauffman conditions for a deadlock to occur are:
* Mutual Exclusion: There is a resource that can only be used by one process at a time
* Hold and Wait: There is a process that needs to hold a resource and wait for others
* No Preemption: The monitor cannot forve a process to give up a resource
* Circular Wait: A waits for B who waits for C who waits for A

<h4>Livelock</h4>
Similar to a deadlock, except that the states of the processes involved constantly change with regard to one another, none progressing

<h4>The Cap Theorem</h4>

Published by Eric Brewer in 2000, the theorem is a set of basic requirements that describe any distributed system. If you imagine a distributed database system with multiple servers, here's how the CAP theorem applies:

* Consistency - All the servers in the system will have the same data so users will get the same copy regardless of which server answers their request.
* Availability - The system will always respond to a request (even if it's not the latest data or consistent across the system or just a message saying the system isn't working).
* Partition Tolerance - The system continues to operate as a whole even if individual servers fail or can't be reached.

It's theoretically impossible to have all 3 requirements met, so a combination of 2 must be chosen and this is usually the deciding factor in what technology is used.

When it comes to distributed databases, the two choices are only AP or CP because if it's not partition tolerant, it's not really a reliable distributed database. So the choice is simpler: if a network split happens, do you want the database to keep answering but with possibly old/bad data (AP)? Or should it just stop responding unless you can get the absolute latest copy (CP)?

------------------

<a id='Fault Tolerance'></a>
<h4 style="background-color:#616161;color:white">Fault Tolerance</h4>

* Many cloud computations run for extended periods of time on multiple servers.
* Checkpoints are taken periodically in anticipation of the need to restart a process when one or more
systems fail.
* When a failure occurs, the computation is restarted from the last checkpoint rather than from the beginning.
* Intuitively, the construction of the global state is equivalent to taking snapshots of individual processes and then combining these snapshots into a global view.

* Yet, combining snapshots is straightforward if and only if all processes have access to a global clock and the snapshots are taken at the same time

<b>Chandy-Lamport Algorithm</b>

1. The observer process (the process taking a snapshot):  
    * Saves its own local state
    * Sends a snapshot request message bearing a snapshot token to all other processes
2. A process receiving the snapshot token for the first time on any message: 
    * Sends the observer process its own saved state
    * Attaches the snapshot token to all subsequent messages (to help propagate the snapshot token)
3. When a process that has already received the snapshot token receives a message that does not bear the snapshot token, this process will forward that message to the observer process. This message was obviously sent before the snapshot “cut off” (as it does not bear a snapshot token and thus must have come from before the snapshot token was sent out) and needs to be included in the snapshot.

From this, the observer builds up a complete snapshot: a saved state for each process and all messages “in the ether” are saved

------------------

<h4>Crash Failures</h4>
1. Crash failure: the process abruptly stops and does not resume
2. Byzantine failure: A Byzantine fault is any fault presenting different symptoms to different observers. A Byzantine failure is the loss of a system service due to a Byzantine fault in systems that require consensus.
    * Omission failures, e.g., failing to receive a request
    or send a response.
    * Commission failures, e.g., corrupting the local state, sending an incorrect/inconsistent response to the request.

Of the two types of failures,Byzantine failures are far more disruptive

<h4>Paxos Consensus Protocol</h4>

18min 44: https://www.youtube.com/watch?v=WX4gjowx45E

The basic Paxos protocol assumes
– Processes run on processors and communicate
through a network;
processors and network may experience failures,
but not Byzantine failures.

The Processors
* operate at arbitrary speeds;
* have stable storage and may rejoin the protocol after a failure;
* send messages to one another

The network:
* may lose, reorder, or duplicate messages;
* messages are sent asynchronously; it may take arbitrary long time to reach the destination

The basic Paxos protocol has two phases.

Phase I
* Proposal preparation
* Proposal promise

Phase II
* Accept request
* Accepted

The benefit of the Paxos protocols is the
guarantee of three safety properties
* Non-triviality: Only proposed values can be
learned.

* Consistency: at most one value can be learned.

* Liveness: if a value v has been proposed,
eventually every learner will learn some value,
provided that sufficient processors remain nonfaulty.

-----------------------

<b>Parallel Computing</b>

The speed-up S measures the effectiveness of parallelization:

S(N) = sequential
computation time of a task / N parallel
computation time of task

<b>Embarrassingly Parallel Problems</b>
*Little or no effort is required to separate the
problem into a number of parallel tasks, i.e.,
$\alpha$ ≈ 0


<b>Amdahl’s Law</b>
* When the problem/dataset size is fixed Amdahl’s Law accounts for the fact that there will still be a non-parallelizable element
– If $\alpha$ = % of sequential time that can't be parallelized:

S(N) = 1 / [$\alpha$ + (1-$\alpha$)/N] < 1 / $\alpha$

* The speedup of a program using multiple processors in parallel computing is limited by the sequential fraction of the program.

<b>Gustafan's Law</b>
S(N) = $\alpha$ + N (1- $\alpha$) = N - $\alpha$ (N-1)

The limitations of the sequential part of a code can be balanced by increasing the problem size (i.e. do more in one go).

<b>Comparison</b>
– Amdahl’s law argues that
more computing power will make an analysis of
the same dataset faster.
– Gustafson’s law argues that
more computing power will make bigger datasets
be analysed or deeper analysis be performed


<b>Example question:</b>
A data mining program consists of four consecutive parts, P1, P2, P3 and P4 with
the percentages of runtime being 10%, 30%, 40% and 20% respectively on a single
processor. It is known that P1 and P3 can be parallelised while P2 and P4 cannot. If
the problem/dataset size is fixed, how much speedup can this program achieve at most
through parallel computing, according to Amdahl’s Law? If the problem/dataset size
can be arbitrarily large, how much speedup can this program achieve at most through
parallel computing, according to Gustafson’s Law?

<h4>Single Program Multiple Data (SPMD)</h4>
Single Program Multiple Data (SPMD):
multiple copies of the same program run
concurrently, each one on a different data block

<h4>Concurrency</h4>
Concurrency is important
* Many cloud applications are data-intensive and
use a number of instances which run concurrently

Concurrency is challenging
* It could lead to race conditions, an undesirable
effect when the results of concurrent execution
depend on the sequence or timing of events
* Shared resources must be protected by locks /
semaphores / monitors to ensure serial access

<h4>Distributed File Systems</h4>

* E.g. Google's GFS, and HDFS for Hadoop
* Data kept in "chunks" spread across machines
* Each chunk replicated on different machines
* Seamless recovery from disk or machine failure

<h4>Assumptions behind distributed file systems</h4>

* Commodity hardware over “exotic” hardware
    * Scale “out”, not “up”
* High component failure rates
    * Inexpensive commodity components fail all the time
* “Modest” number of huge files
    * Multi-gigabyte files are common, if not encouraged
* Files are write-once, mostly appended to
    * Perhaps concurrently
* Large streaming reads over random access
    * High sustained throughput over low latency

<h4>HFDS Indexing & Hashing</h4>
* Assumes a very large Key-Value collection
* The Key-Value index supports insertions, deletions, key searches, range searches, and node leave & join operations.
* Uses a hashing function to map data into a specified number of buckets. servers. This way, you can control the size of the data subsets (i.e., buckets) and optimize for query speed.

A naïve solution
* Let N be the number of servers.
* The function modulo(h(k), N) = i maps a pair (k,v)
to server Si .
* Fact: if N changes, or if a client uses an invalid
value for N, the mapping becomes inconsistent

With consistent hashing, addition or removal of an instance does not significantly change the mapping of keys to servers.

* A simple, non-mutable hash function h maps both the keys and the servers’ IPs to a large address space A (e.g., [0, 264−1]) organized as a ring in clockwise order.
* If S and S′ are two adjacent servers on the ring: all the keys in range [h(S),h(S′)] are mapped to S.
* If a server fails put a copy on the next machine (on the ring)

<h4>Load Balancing</h4>
Map a server to several points on the ring (virtual
nodes)
* the more points, the more load received by a server;
* also useful if the server fails: data relocation is more
evenly distributed.
* also useful in case of heterogeneity (the rule in largescale
systems).

<h4>Hashing Directory</h4>
* Stored on the master node which is therefore not scalable
* Aamzon Dynamo duplicates and maintains a full hash directory at each node via 'gossiping': Queries can be routed to the right server in 1 message

<h4>Relational Databases</h4>
* Multipurpose
    * transactions & analysis
    * Batch & interactive
* Data integrity via ACID transactions
* Lots of tools in software ecosystem
    * for ingesting, reporting, etc.
* Supports SQL (and SQL integration, e.g., JDBC)
* Automatic SQL query optimization

* Projecting means choosing some columns from each record and leaving others out.
    * Easy in Map-Reduce: Map over tuples, emit new tuples with appropriate attributes
    * No reducers
        * unless for regrouping or resorting tuples
    * Speed of encoding/decoding tuples becomes important
    * Relational databases take advantage ofcompression
    * Semi-structured data? No problem

<h4>Hadoop</h4>

MapReduce (Hadoop):
* Designed for large clusters, fault tolerant
* Data is accessed in “native format”
* Supports many query languages
* Programmers retain control over performance
* Open source

Online Transaction Processing (OLTP)
* Typical applications:
    * e-commerce, banking, airline reservations
* User facing:
    * real-time, low latency, highly-concurrent
* Tasks:
    * relatively small set of “standard” transactional queries
* Data access pattern:
    * random reads, updates, writes (involving relatively
small amounts of data

Online Analytical Processing (OLAP)
* Typical applications:
    * business intelligence, data mining
* Back-end processing:
    * batch workloads, less concurrency
* Tasks:
    * complex analytical queries, often ad hoc
* Data access pattern:
    * table scans, large amounts of data involved per query
    
<b>Hadoop is perfect for ETL:</b>
* Most likely, you already have some data
warehousing solution
* Ingestion is limited by the speed of HDFS
* Scales out with more nodes
* Massively parallel
* Ability to use any processing tool
* Much cheaper than parallel databases
* ETL is a batch process anyway!

<a id='Distributed Systems'></a>
<h3 style="background-color:#616161;color:white">2. Map-Reduce</h3>

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud002.png" height="100" width="500">



<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud003.png" height="100" width="400">

<b>Partitioning:</b>
* Used to ensure records with the same key end up at the same worker
* Uses a simple function hash(key) mod R to decide which Reducer to go to
* Sometimes useful to override. E.g hash(hostname(URL)) mod R ensures URLs from one host end up in the same output file

<b>Join query in Map-Reduce</b>

* Reduce-side join: 
    * Map over both tables and emit join key & value
    * Reducer: Group by join key, then perform actual join

* Map-side join: 
    * Assume two datasets are sorted by the join key to start with
    * Partition and sort both datasets in the same manner
    * Map over one dataset, read from other corresponding partition
    * But a consistenly partioned dataset may not be realistic
    
* In-memory join
    * Basic version: Each node has a copy of the lookup table so can do a join during map stage. No reducers needed.
    – Striped variant: Divide lookup table up across nodes so that it can fit in memory. Combine results in reducer later.
    – Memcached variant: In memory key-value store. Very fast. Scales out with cluster.

Limitations of each?
* In-memory join: memory
* Map-side join: sort order and partitioning
* Reduce-side join: general purpose

<h4> Combiners </h4>

This mapper function which has a multii-doc wrapper for a single doc function exemplifies the need for a combiner in a word count of individual documents

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud005.png" height="100" width="400">

* The combiner could group together counts for the same document
* While the combiner can do a similar role to the reducer, bear in mind the combiner only gets a subset of the data, so if you were trying to calculate averages for example you'd end calculating a mini average and pushing that forward to the reducer
* Using the reducer as a combiner works only if the function we're computing is both commutative (a + b = b + a) and associative (a + (b + c) = (a + b) + c). 

http://www.philippeadjiman.com/blog/2010/01/14/hadoop-tutorial-series-issue-4-to-use-or-not-to-use-a-combiner/

<b>Calculating a co-occurence matrix</b>

<b>Pairs method:</b>
<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud006.png" height="100" width="500">

<b>Stripes method:(Essentially you can emit an entire row at a time)</b>
<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud007.png" height="100" width="500">

<h4>Drawback of stripes</h4>
* First, what if the data structure in Reduce exceeds the size of the available memory? This would not happen for the color example, but it might for other problems where the table has many more columns. 
* Second, the granularity of the Reducer workload is limited by the number of different keys. What if we have more machines than keys? How can we keep all of them busy?


<a id='MapReduce Optimization'></a>
<h3 style="background-color:#616161;color:white">4. MapReduce Optimization</h3>

<b>Ways to optimize:</b>
* Compression of data from one flow to the next
    * Trade-off between the volume of data transfer and the (de)compression time.
    * Usually, compressing map outputs using a fast compressor increases efficiency

* Available memory
    * Particularly for Reduce Shuffle and Sort

* Speculative Execution
    * Jobtracker tries to detect tasks that are taking a long time, such as one node being very slow
    * In this case it would make copies of the tasks and execute them
    * Whichever one finishes first wins
    

<a id='AlgoDesign'></a>
<h3 style="background-color:#616161;color:white">5. Map-Reduce Algorithm Design</h3>

* <b>“In-Mapper Combining”</b>
    * The in-mapper combiner takes this optimization a bit further: the aggregations do not even write to local disk: they occur in-memory in the Mapper itself.
    * So one example is to have an Initialize, Map, an Call method in your Mapper, where the intialize setups an an associative array to store results, the mappers does each indicidual processing and stores in the array, and close does an aggregation before emitting

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud005.png" height="100" width="500">

Advantages: Speed as not writing to disk
Disadvantages: Explicit memory management required. Potential for order-dependent bugs

* <b>“Pairs” vs “Stripes”</b>
 * Pairs is just emitting a key and a value
 * Stripes is grouping multiple results into an associative array with a defined structure so thaat in the combiner or reduce phase it can efficiently interpret the results
 
* <b>Secondary Sorting</b>
    * Value-to-Key Conversion: Creating a composite key is straight forward. What we need to do is analyze what part(s) of the value we want to account for during the sort and add the appropriate part(s) to the natural key. 
    
* Order Inversion
    * The design pattern for controlling the order in which intermediate results are computed is called “order inversion.” 
    * The Mapper will emit a dummy along with the value, where the dummy should always come first if sorted
    * There would be an IF statement in the reducer to then calculate the dummy first

<a id='Hadoop'></a>
<h3 style="background-color:#616161;color:white">3. Hadoop</h3>

* SequenceFiles: Binary encoded of a sequence of key/value pairs

* Concrete classes for different data types:
    * IntWritable
    * LongWritable
    * Text
    * etc.

* Writable Defines a de/serialization protocol.
* Every data type in Hadoop is a Writable.
* WritableComparable Defines a sort order.
* All keys must be of this type (but not values).


<b>Job submission process</b>
* Client (i.e., driver program) creates a job, configures it, and submits it to job tracker
* JobClient computes input splits (on client end)
* Job data (jar, configuration XML) are sent to JobTracker
* JobTracker puts job data in shared location, enqueues tasks
* TaskTrackers poll for tasks
* Off to the races…

<h4>Hadoop Map-Reduce Process</h4>
<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud004.png" height="100" width="400">

* Map outputs are buffered in memory in a circular
buffer
* When buffer reaches threshold, contents are
“spilled” to disk
* Spills merged in a single, partitioned file (sorted
within each partition): combiner runs here


Reduce side
* First, map outputs are copied over to reducer
machine
* “Sort” is a multi-pass merge of map outputs
(happens in memory and on disk): combiner runs
here
* Final merge pass goes directly into reducer

<a id='Spark'></a>
<h3 style="background-color:#616161;color:white">6. Spark</h3>

<b>Advantages:</b>
* Speed due to being in memory    
* RDDs allow for data sharing across jobs
* Good for data that comes in batches (e.g. streaming)

<b>Disadvantages:</b>
* Its “in-memory” capability can become a bottleneck when it comes to cost-efficient processing of big (transactional) data.
    
<b>RDD's</b> 
* Resiliant Distributed Datasets
    * A read-only, partitioned collection of records.
    * Held as distributed shared memory
    * Can only be built from a series of deterministic transformations (e.g. map, filter, join)
    * This is known as a DAG (Direct Acyclic Graph)
    * Fault tolerance - RDDs track the graph of transformations that built them (their lineage) to rebuild lost data
    * Spark offers over 80 high-level operators that make it easy to build parallel apps

RDDs can express many existing parallel
models
– MapReduce, DryadLINQ
– Iterative MapReduce
– Pregel
– SQL

<b>Iterative vs. interactive</b>
* Spark makes 'working set' of data a first‐class concept to efficiently support:
    * Iterative algorithms    (many in machine learning)    
    * Interactive data mining tools (R, Excel, Python)    

<b>Operations on RDDs</b>
* Transformations define new RDDs
    * 'map' means a one-to-one mapping
    * 'flatMap' maps each input value to one or more outputs (similar to the map in MapReduce)
* Actions use RDDs to return results to the driver program, or export results to the storage system
    * collect
    * reduce
    * count
    * save
    * lookupKey

* Spark computes RDDs lazily the first time they are used in an action so that it can pipeline transformations. i.e., an RDDs is defined by transformations but its computation is actually triggered by actions

Control of RDDs
* persistence (storage in RAM, on disk, etc)
    * Users can indicate which RDDs they will reuse and choose a storage strategy for them.

* partitioning (layout across nodes)
    * Users can indicate how an RDD’s elements be partitioned across machines based on a key in each record. This is useful for placement optimizations, such as ensuring that two datasets that will be joined together are hash-partitioned in the same way


* Narrow / wide dependency
    * Narrow: Each partition of the parent RDD is used by at most one partition of the child RDD
    * Wide:Each partition of the parent RDD is used by multiple partitions of the child RDD
    * Allows for pipelined execution on one cluster node, while wide dependencies require data from all parent partitions to be available and to be shuffled across the nodes using a MapReduce-like operation.
    * Recovery after a node failure is more efficient with a narrow dependency as only the lost parent partitions need to be recomputed, and they can be recomputed in parallel on different nodes.

* Discretized Stream (D-Stream) Model
    * Run a streaming computation as a series of small, stateless, and deterministic batch computations
    * Same recovery schemes at much smaller timescale
    * Work to make batch size as small as possible
    
<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud009.png" height="100" width="350">

<a id='Q&A'></a>
<h3 style="background-color:#616161;color:white">8. Q&A</h3>

<h4>
There is a large text file of information about films stored in the HDFS over a number of
machines. Each line of this file describes the details of one film in the following format.
title | year | runtime | genres | stars
<br><br>
The different fields are separated by the | character; the list of genres and the list of stars
are both separated by commas; the runtime is measured in minutes.
An example line is given below.
The Godfather | 1972 | 175 | Crime, Drama | Marlon Brando, Al Pacino, James Caan
You can assume that there are no duplicate records, and each distinct star’s name is unique.
Write a MapReduce program (in pseudo-code) to calculate for each genre the average
runtime of film in the 1970s.
A combiner should be implemented to accelerate the computation.
</h4>

<code>

<img src="https://github.com/BadrulAlom/Data-Science-Notes/raw/master/_img/cloud/cloud008.png" height="100" width="400">

---------------


<h4>There is a large text file of computer science bibliography data held in an HDFS over a
number of machines. It is the same file as described in the previous question.
Each line of this file describes the details of one paper in the following format.
authors | title | conference | year
The different fields are separated by the | character, and the list of authors are separated by
commas. An example line is given below.
D Zhang, J Wang, D Cai, J Lu | Self-Taught Hashing for Fast Similarity Search | SIGIR | 2010
You can assume that there are no duplicate records, and each distinct author or conference
has a different name.
Write a MapReduce program (in pseudo-code) to calculate for each author the total number
of papers he/she has published.
The “in-mapper combining” pattern should be implemented to accelerate the computation.</h4>

<h4>There is a large text file of computer science bibliography data held in an HDFS over a
number of machines. It is the same file as described in the previous two questions.
Each line of this file describes the details of one paper in the following format.
authors | title | conference | year
The different fields are separated by the | character, and the list of authors are separated by
commas. An example line is given below.
D Zhang, | Wang, D Cai, | Lu | Self-Taught Hashing for Fast Similarity Search | SIGIR | 2010
You can assume that there are no duplicate records, and each distinct author or conference
has a different name.
Write a MapReduce program (in pseudo-code), using the “stripes” pattern, to calculate for
each author the total number of papers he/she has co-authored with each different collaborator
(i.e., researcher who has co-authored papers with him/her before). For example, the
output corresponding to the author D Zhang could be as follows.
D Zhang — D Cai: 3, WS Lee: 13, | Lu: 6, | Wang: 8
A combine function should be implemented to accelerate the computation.</h4>

<a id='Revision Lecture Notes'></a>
<h3 style="background-color:#616161;color:white">9. Revision Lecture Notes</h3>

* 8 factual questions - keep answers short
* 3 algorithm questions (1 graph based)
* New for this year - Apache Spark
